---
#**LABORATORIO Nº 04:  K-MEANS**
---




# Alumno: Huillca Mozo, Bryan
# Codigo: 160329

# **Práctica de laboratorio 5b: k-Means para cuantificar atributos**

#### Los algoritmos de agrupación de datos, además de utilizarse en el análisis exploratorio para extraer patrones de similitud entre objetos, pueden utilizarse para comprimir el espacio de datos.

#### En este notebook usaremos nuestra base de datos Sentiment Movie Reviews para los experimentos. Primero usaremos la técnica word2vec que aprende una transformación de tokens desde una base a un vector de atributos. A continuación, utilizaremos el algoritmo k-Means para comprimir la información sobre estos atributos y proyectar cada objeto en un espacio de atributos de tamaño fijo.

#### Las celdas de ejercicio comienzan con el comentario `# EJERCICIO` y los códigos a completar están marcados con los comentarios `<COMPLETO>`.

#### ** En este notebook: **
#### *Parte 1:* Word2Vec
#### *Parte 2:* k-Means para cuantificar atributos
#### *Parte 3:* Aplicar un k-NN

### **Parte 0: Preliminares**

#### Para este notebook usaremos la base de datos de reseñas de películas que se usará para el segundo proyecto.

#### La base de datos tiene los campos separados por '\t' y el siguiente formato:
    `"id de frase","id de oración","Frase","Sentimiento"`

#### Para esta práctica de laboratorio solo usaremos el campo "Frase".

# ***Instalacion de dependencias para pyspark***

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

#VERSION PYSPARK

In [2]:
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 33 kB/s 
     |████████████████████████████████| 198 kB 53.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=3e1047de0600efc6aa6e20bd2e85b07cc2f2ec6f3351abcaad0f45f7b88bb912
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


#Uso de Sparksesion y SparkContext

In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[4]").getOrCreate()

In [4]:
import os
import numpy as np

def parseRDD(point):
    """ Parser for the current dataset. It receives a data point and return
        a sentence (third field).
    Args:
        point (str): input data point
    Returns:
        str: a string
    """    
    data = point.split('\t')
    return (int(data[0]),data[2])

def notempty(point):
    """ Returns whether the point string is not empty
    Args:
        point (str): input string
    Returns:
        bool: True if it is not empty
    """   
    return len(point[1])>0

filename = os.path.join("Data","MovieReviews2.tsv")
rawRDD = sc.textFile(filename,100)
header = rawRDD.take(1)[0]

dataRDD = (rawRDD
           #.sample(False, 0.1, seed=42)
           .filter(lambda x: x!=header)
           .map(parseRDD)
           .filter(notempty)
           #.sample( False, 0.1, 42 )
           )

print ('Read {} lines'.format(dataRDD.count()))
print ('Sample line: {}'.format(dataRDD.takeSample(False, 1)[0]))

Read 8528 lines
Sample line: (63295, 'An effortlessly accomplished and richly resonant work .')


### **Parte 1: Word2Vec**

#### La técnica [word2vec][word2vec] aprende a través de una red neuronal semántica una representación vectorial de cada token en un corpus de tal manera que las palabras semánticamente similares son similares en la representación vectorial.

#### PySpark contiene una implementación de esta técnica, para aplicarla basta con pasar un RDD en el que cada objeto representa un documento y cada documento está representado por una lista de tokens en el orden en que aparecen originalmente en el corpus. Después del proceso de entrenamiento, podemos transformar un token usando el método [`transform`](https://spark.apache.org/docs/latest/ml-features) para convertir cada token en una representación vectorial.

#### En este punto, cada objeto en nuestra base estará representado por una matriz de tamaño variable.

[word2vec]: https://code.google.com/p/word2vec/

### **(1a) Generación de RDD a partir de tokens**

#### Use la función de tokenización `tokenize` para generar un RDD `wordsRDD` que contenga listas de tokens de nuestra base de datos original.

In [6]:
# EXERCICIO
import re

split_regex = r'\W+'

stopfile = os.path.join("Data","stopwords.txt")
stopwords = set((sc.textFile(stopfile)).collect())

def tokenize(string):
    """ An implementation of input string tokenization that excludes stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    str_list = re.split(split_regex, string)
    str_list = filter(lambda w: len(w)>0, map(lambda w: w.lower(), str_list))
    return [w for w in str_list if w not in stopwords]   #COMPLETAR = [w for w in str_list if w not in stopwords]

wordsRDD = dataRDD.map(lambda x: tokenize(x[1]))

print (wordsRDD.take(1)[0])

['quiet', 'introspective', 'entertaining', 'independent', 'worth', 'seeking']


In [7]:
# TEST Tokenize a String (1a)
assert wordsRDD.take(1)[0]==[u'quiet', u'introspective', u'entertaining', u'independent', u'worth', u'seeking'], 'lista incorreta!'

### **(1b) Aplicando la transformación word2vec**

#### Cree una plantilla word2vec aplicando el método `fit` al RDD creado en el ejercicio anterior.

#### Para aplicar este método debes hacer un pipeline de métodos, primero ejecutando `Word2Vec()`, luego aplicando el método `setVectorSize()` con el tamaño que queremos para nuestro vector (usa el tamaño 5), seguido de ` setSeed()` para la semilla aleatoria, en caso de experimentos controlados (usaremos 42) y finalmente `fit()` con nuestro `wordsRDD` como parámetro.

In [8]:
# EXERCICIO
from pyspark.mllib.feature import Word2Vec

model = (Word2Vec().setVectorSize(5).setSeed(42).fit(wordsRDD))  #COMPLETAR = setVectorSize(5).setSeed(42).fit(wordsRDD))

print (model.transform(u'entertaining'))
print (list(model.findSynonyms(u'entertaining', 2)))

[-0.13553844392299652,0.03944551944732666,0.03806566819548607,0.08553558588027954,-0.02614559605717659]
[('cgi', 0.989105761051178), ('something', 0.9889155626296997)]


In [29]:
dist = np.abs(model.transform(u'entertaining')-np.array([0.0136831374839,0.00371457682922,-0.135785803199,0.047585401684,0.0414853096008])).mean()
assert dist<1e-6, 'valores incorretos'
assert list(model.findSynonyms(u'entertaining', 1))[0][0] == 'god', 'valores incorretos'

### **(1c) Generando un RDD de arreglos**

#### Como primer paso, necesitamos generar un diccionario donde la clave son las palabras y el valor es el vector que representa esa palabra.

#### Para esto primero generaremos una lista `uniqueWords` que contiene las palabras únicas de las palabras RDD, eliminando aquellas que aparecen menos de 5 veces [$^1$](#1). A continuación, crearemos un diccionario `w2v` donde la clave es un token y el valor es un `np.array` del arreglo transformado de ese token[$^2$](#2).

#### Finalmente, creemos un RDD llamado `vectorsRDD` donde cada registro está representado por una matriz donde cada fila representa una palabra transformada.

In [14]:
# EXERCICIO
uniqueWords = (wordsRDD.flatMap(lambda ws: [(w, 1) for w in ws]).reduceByKey(lambda x,y: x+y).filter(lambda wf: wf[1]>=5).map(lambda wf: wf[0]).collect())
#COMPLETAR = flatMap(lambda ws: [(w, 1) for w in ws]).reduceByKey(lambda x,y: x+y).filter(lambda wf: wf[1]>=5).map(lambda wf: wf[0]).

print ('{} tokens únicos'.format(len(uniqueWords)))

w2v = {}
for w in uniqueWords:
    w2v[w] = model.transform(w)
w2vb = sc.broadcast(w2v)       
print ('Vetor entertaining: {}'.format( w2v[u'entertaining']))

vectorsRDD = (wordsRDD.map(lambda ws: np.array([w2vb.value[w] for w in ws if w in w2vb.value]))) #COMPLETAR = .map(lambda ws: np.array([w2vb.value[w] for w in ws if w in w2vb.value]))
recs = vectorsRDD.take(2)
firstRec, secondRec = recs[0], recs[1]
print (firstRec.shape, secondRec.shape)

3388 tokens únicos
Vetor entertaining: [-0.13553844392299652,0.03944551944732666,0.03806566819548607,0.08553558588027954,-0.02614559605717659]
(5, 5) (10, 5)


In [16]:
# TEST Tokenizing the small datasets (1c)
assert len(uniqueWords) == 3388,  'valor incorreto'
assert np.mean(np.abs(w2v[u'entertaining']-[-0.13553844392299652,0.03944551944732666,0.03806566819548607,0.08553558588027954,-0.02614559605717659]))<1e-6,'valor incorreto'
assert secondRec.shape == (10,5)

### **Parte 2: k-Means para cuantificar atributos**

#### Llegados a este punto, es fácil ver que no podemos aplicar nuestras técnicas de aprendizaje supervisado a esta base de datos:

   * #### La regresión logística requiere un vector de tamaño fijo que represente cada objeto
   * #### k-NN necesita una forma clara de comparar dos objetos, ¿qué métrica de similitud debemos aplicar?
  
#### Para resolver esta situación, realicemos una nueva transformación en nuestro RDD. Primero, aprovechemos el hecho de que dos tokens con un significado similar se asignan a vectores similares para agruparlos en un solo atributo.

#### Al aplicar k-Means a este conjunto de vectores, podemos crear $k$ puntos representativos y, para cada documento, generar un histograma de recuento de tokens en los clústeres generados.

#### **(2a) Agrupando los vectores y creando centros representativos**

#### Como primer paso generaremos un RDD con los valores del diccionario `w2v`. A continuación, aplicaremos el algoritmo `k-Means` con $k = 200$ y $seed = 42$.

In [18]:
# EXERCICIO
from  pyspark.mllib.clustering import KMeans

vectors2RDD = sc.parallelize(np.array(list(w2v.values())),1)
print ('Sample vector: {}'.format(vectors2RDD.take(1)))

modelK = KMeans.train(vectors2RDD, 200, seed=42)   #COMPLETAR = train(vectors2RDD, 200, seed=42)

clustersRDD = vectors2RDD.map(lambda x: modelK.predict(x))
print ('10 first clusters allocation: {}'.format(clustersRDD.take(10)))

Sample vector: [array([-0.07269461,  0.09603201,  0.20506908, -0.03772384,  0.08151765])]
10 first clusters allocation: [5, 136, 37, 12, 145, 66, 63, 84, 140, 66]


In [20]:
# TEST Amazon record with the most tokens (1d)
assert clustersRDD.take(10)==[5, 136, 37, 12, 145, 66, 63, 84, 140, 66], 'valor incorreto'

#### **(2b) Transformación de matriz de datos en vectores cuantificados**

#### El siguiente paso es transformar nuestro RDD de frases en un RDD de pares (id, vector cuantificado). Para ello crearemos una función cuantificadora que recibirá como parámetros el objeto, el modelo k-means, el valor de k y el diccionario word2vec.

#### Para cada punto, separemos el id y apliquemos la función `tokenize` a la cadena. Luego transformamos la lista de tokens en una matriz word2vec. Finalmente, aplicamos cada vector de esta matriz al modelo k-Means, generando un vector de tamaño $k$ donde cada posición $i$ indica cuántos tokens pertenecen al clúster $i$.

In [22]:
# EXERCICIO
def quantizador(point, model, k, w2v):

    key = point[0]             #COMPLETAR = point[0]   

    words = tokenize(point[1]) #COMPLETAR = tokenize(point[1])

    matrix = np.array( [w2v[w] for w in words if w in w2v] ) #COMPLETAR = [w2v[w] for w in words if w in w2v]

    features = np.zeros(k)
    for v in matrix:
        c = model.predict(v)   #COMPLETAR = model.predict(v) 
        features[c] += 1
    return (key, features)
    
quantRDD = dataRDD.map(lambda x: quantizador(x, modelK, 500, w2v))

print (quantRDD.take(1))

[(64, array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 

In [23]:
# TEST Implement a TF function (2a)
assert quantRDD.take(1)[0][1].sum() == 5, 'valores incorretos'